Test du projet de class trajets indirect

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import traitement_od as t
import pandas as pd
import numpy as np
import altair as alt
import datetime as dt
pd.set_option('display.max_colwidth', 1000)
import os
from sklearn.cluster import DBSCAN

In [549]:
df_journee=t.ouvrir_fichier_lapi('2019-01-29 00:00:00','2019-01-30 23:59:59')

In [714]:
df_journee.loc[(df_journee['camera_id']==19)]

,camera_id,created,immat,fiability,l,state
573617,19,2019-01-29 00:00:42,12bae96beacfdbf21b90e1464adf2c50c54839bb,89,0,ES
573618,19,2019-01-29 00:06:26,6fa3a994405b689483307b360a3cc4243499756a,99,0,ES
573619,19,2019-01-29 00:07:51,57a2d1b6e98cb8a1f013acfddd4624aa1595e5c2,59,0,FR
573620,19,2019-01-29 00:11:16,a4d5982416accf1a99b677ff12018885c5a72611,49,0,ES
573621,19,2019-01-29 00:18:33,146ed30e604fb0e52732fdb487277b735cf738cd,99,0,RO
573622,19,2019-01-29 00:22:34,c47d68409d34472a00f1eb9a13f034abc7a9fd5a,99,0,PT
573623,19,2019-01-29 00:22:56,522d17ded2379ce706c03d35d42bc5cc95b02998,79,0,ES
573624,19,2019-01-29 00:25:11,98d06dcc7f902ca4caf4dac6f15f88d822c96a6e,99,0,ES
573625,19,2019-01-29 00:27:07,2876909b29b2793c2af3b7d6d46a7042eaf815d2,99,0,ES
573626,19,2019-01-29 00:27:14,868b8807b7b68f7d381ac80aad667e9747c22c58,99,0,ES


In [567]:
liste_trajet_locale=(pd.DataFrame([{'origine':'A63','destination':'A10','cam_o':14, 'cam_d':11, 'trajets':[
                                                        {'cameras':[14,19,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A63','cam_o':12, 'cam_d':13,'trajets':[{'cameras':[12,6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'N10','cam_o':14, 'cam_d':5,'trajets':[{'cameras':[14,19,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A63','cam_o':6, 'cam_d':13,'trajets':[{'cameras':[6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A10','cam_o':10, 'cam_d':11,'trajets':[{'cameras':[10,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A62','cam_o':12, 'cam_d':9,'trajets':[{'cameras':[12,6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'N10','cam_o':10, 'cam_d':5,'trajets':[{'cameras':[10,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[10,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A62','cam_o':6, 'cam_d':9,'trajets':[{'cameras':[6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[6,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A62','cam_o':14, 'cam_d':9,'trajets':[{'cameras':[14,19,9],'type_trajet':'indirect'},
                                                        {'cameras':[14,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A63','cam_o':10, 'cam_d':13,'trajets':[{'cameras':[10,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[10,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A63','cam_o':8 ,'cam_d':13,'trajets':[{'cameras':[8,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A89','cam_o':14, 'cam_d':7,'trajets':[{'cameras':[14,19,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,7],'type_trajet':'indirect'},
                                                        {'cameras':[14,7],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A62','cam_o':8, 'cam_d':9,'trajets':[{'cameras':[8,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[8,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'A89','cam_o':10, 'cam_d':7,'trajets':[{'cameras':[10,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[10,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'A10','cam_o':8, 'cam_d':11,'trajets':[{'cameras':[8,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[8,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A89','cam_o':12, 'cam_d':7,'trajets':[{'cameras':[12,6,7],'type_trajet':'indirect'},
                                                        {'cameras':[12,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'N10','cam_o':8, 'cam_d':5,'trajets':[{'cameras':[8,5],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'N10','destination':'A89','cam_o':6, 'cam_d':7,'trajets':[{'cameras':[6,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A630','cam_o':12, 'cam_d':18,'trajets':[{'cameras':[12,6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A630','destination':'A10','cam_o':19, 'cam_d':18,'trajets':[{'cameras':[19,4,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,11],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A630','destination':'A62','cam_o':19, 'cam_d':9,'trajets':[{'cameras':[19,9],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A62','destination':'A630','cam_o':10, 'cam_d':18,'trajets':[{'cameras':[10,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A630','destination':'A89','cam_o':19, 'cam_d':7,'trajets':[{'cameras':[19,4,7],'type_trajet':'indirect'},
                                                         {'cameras':[19,7],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A89','destination':'A630','cam_o':8, 'cam_d':18,'trajets':[{'cameras':[8,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[8,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'N10','destination':'A630','cam_o':6, 'cam_d':18,'trajets':[{'cameras':[6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A630','destination':'N10','cam_o':19, 'cam_d':5,'trajets':[{'cameras':[19,1,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,5],'type_trajet':'direct'},
                                                        ]}
                           ]))[['origine', 'destination', 'cam_o', 'cam_d','trajets']]

In [690]:
#trouver o et d selon num_camera
liste_trajet_od=([tuple(cam['cameras']) for camera in liste_trajet_locale.loc[(liste_trajet_locale['cam_d']==(12)) & 
                                                                              (liste_trajet_locale['cam_o']==(14))].trajets for cam in camera])
trajet_max=[]
for trajet in liste_trajet_od : 
    if len(trajet)==max([len(trajet) for trajet in liste_trajet_od]) : 
        trajet_max.append(trajet)
trajet_max

[]

In [661]:
for index, value in liste_trajet_locale.iterrows() :
    print(index,value[0],value[1],value[2], value[3])

0 A63 A10 14 11
1 A10 A63 12 13
2 A63 N10 14 5
3 N10 A63 6 13
4 A62 A10 10 11
5 A10 A62 12 9
6 A62 N10 10 5
7 N10 A62 6 9
8 A63 A62 14 9
9 A62 A63 10 13
10 A89 A63 8 13
11 A63 A89 14 7
12 A89 A62 8 9
13 A62 A89 10 7
14 A89 A10 8 11
15 A10 A89 12 7
16 A89 N10 8 5
17 N10 A89 6 7
18 A10 A630 12 18
19 A630 A10 19 18
20 A630 A62 19 9
21 A62 A630 10 18
22 A630 A89 19 7
23 A89 A630 8 18
24 N10 A630 6 18
25 A630 N10 19 5


In [668]:
test=t.trajet(df_journee, '2019-01-29 00:00:00', 60, 16, [6,13], type='Global')

[(6, 2, 18, 13), (6, 3, 18, 13)]


PasDePlError: pas de PL sur la période et les cameras visées

In [ ]:
test=t.transit_1_jour(df_journee,'2019-01-29 00:00:00', liste_trajet_locale)

DatetimeIndex(['2019-01-29 00:00:00', '2019-01-29 01:00:00',
               '2019-01-29 02:00:00', '2019-01-29 03:00:00',
               '2019-01-29 04:00:00', '2019-01-29 05:00:00',
               '2019-01-29 06:00:00', '2019-01-29 07:00:00',
               '2019-01-29 08:00:00', '2019-01-29 09:00:00',
               '2019-01-29 10:00:00', '2019-01-29 11:00:00',
               '2019-01-29 12:00:00', '2019-01-29 13:00:00',
               '2019-01-29 14:00:00', '2019-01-29 15:00:00',
               '2019-01-29 16:00:00', '2019-01-29 17:00:00',
               '2019-01-29 18:00:00', '2019-01-29 19:00:00',
               '2019-01-29 20:00:00', '2019-01-29 21:00:00',
               '2019-01-29 22:00:00', '2019-01-29 23:00:00'],
              dtype='datetime64[ns]', freq='H')
date : 2019-01-29 00:00:00 debut_traitement : 2019-04-03 17:34:26.071201
date : 2019-01-29 01:00:00 debut_traitement : 2019-04-03 17:34:42.072935
date : 2019-01-29 02:00:00 debut_traitement : 2019-04-03 17:34:58.319148


In [709]:
#MATRICE O-D !!!
pd.pivot_table(test,values='l', index='origine', columns='destination',aggfunc='count' )

destination,A10,A62,A63,A630,A89,N10
origine,,,,,,
A10,NaN,32.0,21.0,32.0,7.0,NaN
A62,19.0,NaN,NaN,19.0,6.0,22.0
A63,NaN,NaN,NaN,NaN,1.0,NaN
A630,NaN,29.0,NaN,NaN,30.0,404.0
A89,7.0,12.0,5.0,39.0,NaN,25.0
N10,NaN,65.0,151.0,87.0,24.0,NaN


In [716]:
test.loc[test['origine']=='A63']

,date_cam_1,immat,l,cameras,date_cam_2,tps_parcours,o_d,origine,destination
261,2019-01-29 10:45:24,7569b9bc2d302c23c4c681cf712b8719009463b0,1,"(14, 19, 4, 7)",2019-01-29 11:51:24,01:06:00,A63-A89,A63,A89


,date_cam_1,immat,l,cameras,date_cam_2,tps_parcours
475,2019-01-29 08:15:57,565319bccae6af27bea3882d6950954f110c08ea,1,"(8, 5)",2019-01-29 08:46:29,00:30:32
548,2019-01-29 08:18:45,408b6da06c70d3e0e633aa9bc167b63f86118a95,1,"(8, 5)",2019-01-29 08:42:10,00:23:25
680,2019-01-29 08:23:12,f0315689d509f01da93b128ab687d4b506806a00,1,"(8, 5)",2019-01-29 08:51:41,00:28:29
721,2019-01-29 08:24:21,55073f56d0f5699e47cfe4108522eca7d7058ad8,1,"(8, 5)",2019-01-29 11:47:19,03:22:58
1024,2019-01-29 08:35:19,f27057ee9647b01ecd9d79d3c6ed520a5a942499,1,"(8, 5)",2019-01-29 13:05:30,04:30:11
1119,2019-01-29 08:38:40,7466c759dfa6d61602fa520744b487a00e441ad8,1,"(8, 5)",2019-01-29 10:49:02,02:10:22
1183,2019-01-29 08:41:14,c1e16a598c646e59e60dd80eaf4fb7de77ea4b4e,1,"(8, 5)",2019-01-29 12:30:22,03:49:08
1361,2019-01-29 08:48:20,ae7dbeba189d80635617310239ed620243322fd3,1,"(8, 5)",2019-01-29 11:14:07,02:25:47
1468,2019-01-29 08:52:12,93d2f35c3325cc48feaf8264c7316a8e8f0c746e,1,"(8, 5)",2019-01-29 10:04:23,01:12:11
